# 基于高德地图API获取城市公交站点和线路  


## 导入模块

In [1]:
import requests
import pandas as pd
import json
import re
import time
from bs4 import BeautifulSoup
import math

C:\Users\zhong'shuai\AppData\Local\Temp\ipykernel_31504\884600939.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 设置高德地图API秘钥

In [2]:
key = '55cca19fe666fe4297cad49e4859b4d7'  # 填写自己的秘钥

## 获取首字母

In [3]:
def getInitial(cityName,headers):
    url = 'https://{}.8684.cn/list1'.format(cityName)
    headers = {'User-Agent':headers}
    data = requests.get(url,headers=headers)
    soup = BeautifulSoup(data.text, 'lxml')
    initial = soup.find_all('div',{'class':'tooltip-inner'})[3]
    initial = initial.find_all('a')
    ListInitial = []
    for i in initial:
        ListInitial.append(i.get_text())
    return ListInitial

def getLine(cityName,n,headers,lines):
    url = 'https://{}.8684.cn/list{}'.format(cityName,n)
    headers = {'User-Agent':headers}
    data = requests.get(url,headers=headers)
    soup = BeautifulSoup(data.text, 'lxml')
    busline = soup.find('div',{'class':'list clearfix'})
    busline = busline.find_all('a')
    for i in busline:
        lines.append(i.get_text())

## 坐标数据转换

In [4]:
# 公交坐标数据转化
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率
 
def gcj02towgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + 0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 * math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 * math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 * math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret
 
def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + 0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 * math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 * math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 * math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    if lng < 72.004 or lng > 137.8347:
        return True
    if lat < 0.8293 or lat > 55.8271:
        return True
    return False

def coordinates(c):
    lng,lat = c.split(',')
    lng,lat = float(lng),float(lat)
    wlng,wlat = gcj02towgs84(lng,lat)
    return wlng,wlat

## 爬取公交站点信息

In [5]:
def get_dt(city,line):
    url = 'https://restapi.amap.com/v3/bus/linename?s=rsv3&extensions=all&key='+key+'&output=json&city={}&offset=2&keywords={}&platform=JS'.format(city,line)
    r = requests.get(url).text
    rt = json.loads(r)
    try:
        if rt['buslines']:
            if len(rt['buslines']) == 0:  #有名称没数据
                print('no data in list..')
            else:
                du = []
                for cc in range(len(rt['buslines'])):
                    dt = {}
                    dt['line_name'] = rt['buslines'][cc]['name'] 
                    st_name = []
                    st_coords = []
                    st_sequence = []
                    for st in rt['buslines'][cc]['busstops']:
                        st_name.append(st['name'])
                        st_coords.append(st['location'])
                        st_sequence.append(st['sequence'])
    
                    dt['station_name'] = st_name
                    dt['station_coords'] = st_coords
                    dt['sequence'] = st_sequence
                    du.append(dt)                
                dm = pd.DataFrame(du)
                return dm
        else:
            pass
    except:
        print('error..try it again..')
        time.sleep(2)
        get_dt(city,line)

## 获取公交线路数据信息

In [6]:
def get_line(city,line):
    url = 'https://restapi.amap.com/v3/bus/linename?s=rsv3&extensions=all&key='+key+'&output=json&city={}&offset=2&keywords={}&platform=JS'.format(city,line)
    r = requests.get(url).text
    rt = json.loads(r)
    try:
        if rt['buslines']:
            if len(rt['buslines']) == 0:  #有名称没数据
                print('no data in list..')
            else:
                du = []
                for cc in range(len(rt['buslines'])):
                    dt = {}
                    dt['line_name'] = rt['buslines'][cc]['name'] 
                    dt['polyline'] = rt['buslines'][cc]['polyline']
                    du.append(dt)
                dm = pd.DataFrame(du)
                return dm
        else:
            pass
    except:
        print('error..try it again..')
        time.sleep(2)
        get_dt(city,line)
 
# 爬取公交数据主程序
def get_station_line_inf(citys,chinese_city_names,headers,file_path):
    #对citys列表中的每一个城市获取公交线路名称
    for city,city_name in zip(citys,chinese_city_names):
        #创建公交线路空列表
        lines = []
        #爬取首字母列表，返回首字母列表ListInitial
        ListInitial = getInitial(city,headers)
        #根据首字母列表for循环爬取1-Z首字母线路，append入全局变量lines
        for n in ListInitial:
            getLine(city,n,headers,lines)
        print('正在爬取{}市的公交线路名称...'.format(city_name))
        # save_path
        station_file = file_path + city + '_station.csv'
        line_file = file_path + city +'_line.csv'
    
        # 爬取公交站点数据
        times = 1
        print('正在爬取{}市的公交站点数据...'.format(city_name))
        for sta in lines:
            if times == 1:
                data=get_dt(city_name,sta)
                times += 1
            else:
                dm=get_dt(city_name,sta)
                data=pd.concat([data,dm],ignore_index=True)
                
        for i in range(data.shape[0]):
            coord_x=[]
            coord_y=[]
            for j in data['station_coords'][i]:
                coord_x.append(eval(re.split(',',j)[0]))
                coord_y.append(eval(re.split(',',j)[1]))
            a=[[data['line_name'][i]]*len(data['station_coords'][i]),coord_x,coord_y,data['station_name'][i],data['sequence'][i]]
            df=pd.DataFrame(a).T
            if i==0:
                df1=df
            else:
                df1=pd.concat([df1,df],ignore_index=True)
        df1.columns=['line_name','coord_x','coord_y','station_name','sequence']
        
        # 将公交站点坐标转化为WGS84
        print('正在将{}公交站点数据进行坐标转换...'.format(city_name))
        t_x=[]
        t_y=[]
        for i in range(len(list(df1['coord_x']))):
            [X,Y]=gcj02towgs84(list(df1['coord_x'])[i],list(df1['coord_y'])[i])
            t_x.append(X)
            t_y.append(Y)
        t_x=pd.DataFrame(t_x)
        t_y=pd.DataFrame(t_y)
        df1['coord_x']=t_x
        df1['coord_y']=t_y
        df1.to_csv(station_file,index=None,encoding='utf_8_sig') 
        
        # 爬取公交线路数据
        times = 1
        print('正在爬取{}市的公交线路数据...'.format(city_name))
        for sta in lines:
            if times == 1:
                data=get_line(city_name,sta)
                times += 1
            else:
                dm=get_line(city_name,sta)
                data=pd.concat([data,dm],ignore_index=True)
        
        # 将公交线路坐标转化为Wgs84
        print('正在将{}公交线路数据进行坐标转换...'.format(city_name))
        name = []
        lons = []
        lats = []
        orders = []
        for uu in range(len(data)):
            linestr = [coordinates(c) for c in data['polyline'][uu].split(';')]
            for m in range(len(linestr)):
                name.append(data['line_name'][uu])
                orders.append(m)
                lons.append(linestr[m][0])
                lats.append(linestr[m][1])
        dre = {'line_name': name, 'lon': lons, 'lat':lats, 'orders':orders}
        data = pd.DataFrame(dre)
        data.to_csv(line_file,index=None,encoding='utf_8_sig')

## 获取城市公交站点和线路

In [8]:
# 城市列表（拼音和中文，支持多个城市）
citys = ['changzhi']  
chinese_city_names = ['长治']
# 浏览器头文件
headers = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36' # 浏览器user-agnet
# 设置保存路径
file_path = '/home/mw/project/' 
get_station_line_inf(citys,chinese_city_names,headers,file_path)

正在爬取长治市的公交线路名称...
正在爬取长治市的公交站点数据...
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
error..try it again..
